In [1]:
import copy
import numpy as np

# Supervised Machine Learning: Regression and Classification 

## Week 2: Regression with multiple input variables

### Multiple Features

We previoulsy considered a single feature in our linear regression.

We can expand to multiple features, *e.g.* instead of a single feature, $x$, we
have multiple features, $x_1, x_2, ...$.

We use $j$ to index the features, *i.e* the $j^{th}$ feature is $x_j$.

For a given example, $i$, we thus replace our previous scalar, $x^{(i)}$, with
a **row vector**, $\vec{x}^{(i)}$.

The linear formula thus changes from $f_{w,b}(x) = wx + b$ to:
$$f_{\vec{w},b}(\vec{x}) = \vec{w}\cdot\vec{x} + b$$
Note that we are computing the dot product of $\vec{w}$ and $\vec{x}$. For these
two row vectors, this is equivalent to pairwise multiplication of the elements
of $\vec{w}$ and $\vec{x}$.

Note that $b$ remains a scalar.

We now have the formula for **multiple linear regression** (which is not the
same thing as *multivariate regression*).

We now have the cost function:
$$J(\vec{w}, b)$$

So the gradient descent algorithm becomes:
$$w_j = w_j - \alpha\frac{\partial}{\partial w_j} J (\vec{w}, b)$$
$$b = b - \alpha\frac{\partial}{\partial b} J (\vec{w}, b)$$

Substituting in the formula for $J$ and differentiating we get the following
algorithm for **gradient descent for multiple linear regression**:

repeat{
$$w_j = w_j - \alpha \frac{1}{m} \sum_{i=1}^{m}
(f_{\vec{w},b}(\vec{x}^{(i)})-y^{(i)})x_j^{(i)} \quad \forall j \in (1, 2, ..., n)$$
$$b = b - \alpha \frac{1}{m} \sum_{i=1}^{m}
(f_{\vec{w},b}(\vec{x}^{(i)})-y^{(i)})$$
... simultaneously update $w_j \forall j \in (1, 2, ..., n)$ and $b$ 
}

Note that the formula for $w_j$ contains both $\vec{x}^{(i)}$ and $x_j^{(i)}$

### An alternative to gradient descent
The **normal equation** can be used as an alternative method to solve for
$\vec{w}$ and $b$ for linear regression *only* (it is not suitable for other
applications). The normal equation has the benefit of not requring iterations,
but it can be slow when the number of feaures is large (*e.g.* above 10,000).
It will not be discussed further, but it may be used under the hood when using
a mature library for performing linear regression.

We will now implement multiple regression in python for the training data below:

| Size (sqft) | Number of Bedrooms | Number of floors | Age of Home | Price (1000s dollars) |
|-----|---|---|----|-----|
|2104 | 5 | 1 | 45 | 460 |
|1416 | 3 | 2 | 40 | 232 |
| 852 | 2 | 1 | 35 | 178 |

In [ ]:
def predict(X: np.ndarray, w: np.ndarray, b: float) -> np.ndarray:
    """Get f(w, b) for an array of X-values.

    Args:
        X: np.ndarray: A 2D matrix of input variables with dimensions
          m x n, where m is the number of rows (observations) and n is
          the number of columns (features).
        w: np.ndarray: A vector of weights, of length n, where n is the
          number of features. 
        b: float: Bias.

    Returns:
        np.ndarray: A vector of length m of predicted values.
    """
    return np.dot(X, w) + b


def cost(X: np.ndarray, y: np.ndarray, w: np.ndarray, b: float) -> np.float64:
    """Compute mean squared error cost function.

    Args:
        X: np.ndarray: A 2D matrix of input variables with dimensions
          m x n, where m is the number of rows (observations) and n is
          the number of columns (features).
        y: np.ndarray: A vector of output variables of length m.
        w: np.ndarray: A vector of weights, of length n, where n is the
          number of features. 
        b: float: Bias.

    Returns:
        np.float64: The MSE cost.
    """
    return np.sum((predict(X, w, b) - y) ** 2) / (2 * X.shape[0])


def gradient(
    X: np.ndarray, y: np.ndarray, w: np.ndarray, b: float
) -> tuple[np.ndarray, np.float64]:
    """Get partial differentials of J_(w, b)(X) with respect to w and b.

    Args:
        X: np.ndarray: A 2D matrix of input variables with dimensions
          m x n, where m is the number of rows (observations) and n is
          the number of columns (features).
        y: np.ndarray: A vector of output variables of length m.
        w: np.ndarray: A vector of weights, of length n, where n is the
          number of features. 
        b: float: Bias.

    Returns:
        tuple[np.ndarray, np.float64]: A tuple of length 2 containing
          the partial derivatives of the MSE cost. The first item in
          the tuple is a vector of length n of the partial derivatives
          with respect to all the weights in w, and the second item is 
          the partial derivative with respect to the bias, b.
    """

    m = X.shape[0]
    error_vector = predict(X, w, b) - y
    dj_dw = np.dot(X.T, error_vector) / m
    dj_db = np.sum(error_vector) / m
    return dj_dw, dj_db


def gradient_descent(
    X: np.ndarray,
    y: np.ndarray,
    w0: np.ndarray,
    b0: float,
    alpha: float,
    max_iters: int,
) -> tuple[np.ndarray, float]:
    """Perform iterative gradient descent.

    Args:
        X: np.ndarray: A 2D matrix of input variables with dimensions
          m x n, where m is the number of rows (observations) and n is
          the number of columns (features).
        y: np.ndarray: A vector of output variables of length m.
        w0: np.ndarray: A vector of length n of initial weights (slopes)
          for each feature.
        b0: float: Bias (intercept).
        alpha: float: Learning rate.
        max_iters: Maximum number of iterations.

    Returns:
        tuple[np.ndarray, np.float64]: Updated values for w and b.
    """
    w = copy.deepcopy(w0)  # avoid modifying external w0.
    b = b0
    for _ in range(max_iters):
        dj_dw, dj_db = gradient(X, y, w, b)
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
    return w, b


X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])
b_init = 785.1811367994083
w_init = np.array([0.39133535, 18.75376741, -53.36032453, -26.42131618])
alpha = 5e-7
max_iters = 1000
predict(X_train, w_init, b_init)

array([459.99999762, 231.99999837, 177.99999899])

### Feature scaling
